In [1]:
from lxml import etree
import pymysql
import time
import re
import pandas as pd

In [2]:
def fast_iter(context, func, func1, *args, **kwargs):

    for event, elem in context:
        
        func(elem, func1, *args, **kwargs)
        # reset elements，clear the internal data of the element
        elem.clear()

    del context
    

In [3]:
def compare(list1,list2):
    if len(list1) < len(list2):
        list1.append('NA')
    return list1

In [12]:
def process_element_contact(elem,func1):

    id_list = []
    name_list =[]
    email_list =[]
    organization_list = []
    lab_list = []
    role_list = []
    

    for contacts in elem.xpath('.//contactInfo'):
        #print(contacts.attrib.values()[0])
        ids = contacts.attrib.values()[0]
        id_list.append(ids)
        for names in contacts.xpath('.//name'):
            name_list.append(names.text)
        for emails in contacts.xpath('.//email'):
            email_list.append(emails.text) 
        for organizations in contacts.xpath('.//organization'):
            organization_list.append(organizations.text)
        for labs in contacts.xpath('.//lab'):
            lab_list.append(labs.text)
        for roles in contacts.xpath('.//role'):
            role_list.append(roles.text)
    
    name_list = func1(name_list,id_list)
    email_list = func1(email_list,id_list)
    organization_list = func1(organization_list,id_list)
    lab_list = func1(lab_list,id_list)
    role_list = func1(role_list,id_list)
    
    contact_info_df = pd.DataFrame(columns=['id', 'name','email','organization','lab','role'])
    contact_info_df['id'] = id_list
    contact_info_df['name'] = name_list
    contact_info_df['email'] = email_list
    contact_info_df['organization'] = organization_list
    contact_info_df['lab'] = lab_list
    contact_info_df['role'] = role_list
    outputpath='contact_info.csv'
    contact_info_df.to_csv(outputpath,sep=',',index=False,header=True)
    return contact_info_df

In [13]:
if __name__ == '__main__':
    print('start', time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))
    start = time.time()


    # By iteratively reading xml, the namespace with the namespace should be added
    context_contact = etree.iterparse('tt.xml',events=('end',),encoding='UTF-8',tag='contactInfoList')
    #context_trail = etree.iterparse('tt.xml',events=('end',),encoding='UTF-8',tag='targetTrack')
    # Quickly read xml data
    fast_iter(context_contact, process_element_contact, compare)
    #fast_iter(context_protocol, process_element_protocol, compare)

    print('stop', time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))
    print('time', time.time()-start)

start 2022-01-18 12:45:31
stop 2022-01-18 12:47:00
time 88.72323298454285
